In [1]:
import mysql.connector
import sshtunnel

import logging
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gensim import corpora
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, precision_recall_curve

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
tunnel = sshtunnel.SSHTunnelForwarder(
        ('13.59.84.252', 22),
        ssh_username='ubuntu',
        ssh_pkey="/Users/kevinmorrill/Downloads/kittenbear.pem",
        remote_bind_address=('constellation2.cluster-c0wmrawlwy5o.us-east-2.rds.amazonaws.com', 3306)
)

print('connected to through tunnel')
tunnel.start()

print(tunnel.local_bind_port)
cnx = mysql.connector.connect(
    user='emo',
    password='kittenbear',
    connection_timeout=10000,
    host='localhost',
    database='libra',
    port=tunnel.local_bind_port)

connected to through tunnel
61283


In [3]:
def read_dataset():
    print('called read dataset')
    posts_raw = []
    labels_raw = []
    
    offset = False
    
    query = "SELECT posts.text, editorial.on_topic FROM posts JOIN editorial ON posts.id = editorial.post_id WHERE editorial.on_topic IS NOT NULL AND LENGTH(posts.text) > 50 AND posts.title IS NOT NULL"
    cursor = cnx.cursor(buffered=True)
    cursor.execute(query)
    
    for row in cursor:
        posts_raw.append(row[0])
        labels_raw.append(row[1])
    
    return posts_raw, labels_raw

posts_raw, labels_raw = read_dataset()

called read dataset


In [4]:
leng = 0
if leng != 0:
    posts_raw = posts_raw[:leng]
    labels_raw = labels_raw[:leng]
len(posts_raw)


865

In [5]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
    for document in posts_raw]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
    for text in texts]

In [6]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(23507 unique tokens: ['(albeit', '(in', '(the', '2007', '2014']...)


In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]

from gensim.matutils import corpus2csc

m = corpus2csc(corpus).transpose()
m.get_shape()

(865, 23507)

In [8]:
def train_classifier(m, training_labels):
    logging.info("Classifier training")
    model = GradientBoostingClassifier(
        n_estimators = 500,
        max_depth = 2
    )
    model = LogisticRegression()

    model.fit(m, np.array(training_labels))
    training_predictions = model.predict(m)
    logging.info('Training predicted classes: {}'.format(np.unique(training_predictions)))
    logging.info('Training accuracy: {}'.format(accuracy_score(training_labels, training_predictions)))
    print(classification_report(training_labels, training_predictions))
    logging.info('Training F1 score: {}'.format(f1_score(training_labels, training_predictions, average='weighted')))
    return model

posts_series = pd.Series(posts_raw)
labels_series = pd.Series(labels_raw)
    
# TODO consider getting more negative data, since we're non representative
x_train, x_test, y_train, y_test = train_test_split(m, labels_raw, random_state=0, test_size=0.2)

model = train_classifier(x_train, y_train)

             precision    recall  f1-score   support

          0       0.99      1.00      0.99       363
          1       1.00      0.99      0.99       329

avg / total       0.99      0.99      0.99       692



In [9]:
def my_predict(model, val):
    if model.predict_proba(val)[0][0] < 0.5:
        return 1
    else:
        return 0

def predict_vec(model, vec):
    return [my_predict(model, v) for v in vec]

def test_classifier(classifier, test_vectors, testing_labels):
    logging.info("Classifier testing")
    testing_predictions = predict_vec(classifier, test_vectors)
    logging.info('Testing predicted classes: {}'.format(np.unique(testing_predictions)))
    print(classification_report(testing_labels, testing_predictions))
    logging.info('Testing accuracy: {}'.format(accuracy_score(testing_labels, testing_predictions)))
    logging.info('Testing F1 score: {}'.format(f1_score(testing_labels, testing_predictions, average='weighted')))
    

test_classifier(model, x_test, y_test)

             precision    recall  f1-score   support

          0       0.77      0.67      0.72        98
          1       0.63      0.73      0.68        75

avg / total       0.71      0.70      0.70       173



from previous run with max/kevin together:

             precision    recall  f1-score   support

          0       0.65      0.66      0.65       181
          1       0.61      0.61      0.61       163

avg / total       0.63      0.63      0.63       344

In [10]:
len(y_test)

173

In [11]:
v = []
cnt = 0
for i in range(0, len(posts_raw)):
    v = m[i]
    res = model.predict(v) 
    if res != labels_raw[i]:
        print("\n\n\nMISPRED %d" % res)
        print(posts_raw[i][:500])
        cnt += 1
        
cnt




MISPRED 1
Take the State of JavaScript 2018 Survey!

Just like Christmas or the flu, the State of JavaScript survey comes back around every year. But unlike these, it’s something to actually look forward to!

If you’re not familiar, the survey covers all things JavaScript and tries to answer two main questions:

Which up-and-coming tools are developers most excited about?

Which tried-and-true technologies are developers most happy with?

These are both equally important: you want to keep up to date with 



MISPRED 1
We’ll be close to capacity for the main expo at the 2019 SaaStr Annual by the end of the month, so we’re going to open up an annex on the lower floor to accomodate more sponsors.

The main expo will be packed. It will be sandwiched right in between the 2 largest stages, with our mentoring lounge right in the middle, and the evening cocktail parties will be centered there, so it will have constant traffic from dusk until dawn.

But the downstairs / secondary expo will b

MISPRED 0
There's a lot of confusing terms in the Cloud space. And that's not counting the term "Cloud." ;)

IaaS (Infrastructure as a Services) - Virtual Machines and stuff on demand.

PaaS (Platform as a Service) - You deploy your apps but try not to think about the Virtual Machines underneath. They exist, but we pretend they don't until forced.

SaaS (Software as a Service) - Stuff like Office 365 and Gmail. You pay a subscription and you get email/whatever as a service. It Just Works.

"Serverless Com



MISPRED 0
It’s well known that SHA-1 is no longer considered a secure cryptographic hash function. Researchers now believe that finding a hash collision (two values that result in the same value when SHA-1 is applied) is inevitable and likely to happen in a matter of months. This poses a potential threat to trust on the web, as many websites use certificates that are digitally signed with algorithms that rely on SHA-1. Luckily for everyone, finding a hash collision is not enough to

56

In [ ]:
query = "SELECT posts.id, posts.text FROM posts JOIN feeds ON posts.feed_id = feeds.id LEFT JOIN editorial ON posts.id = editorial.post_id LEFT JOIN analysis ON posts.id = analysis.post_id WHERE feeds.category_id = 1 AND posts.text IS NOT NULL AND LENGTH(posts.text) > 50 AND analysis.topic_score IS NULL AND editorial.on_topic IS NULL AND posts.created_at > '2018-10-01'"
cursor = cnx.cursor(buffered=True)
cursor.execute(query)
    
for post in cursor:
    stoplist = set('for a of the and to in'.split())
    text = [word for word in post[1].lower().split() if word not in stoplist]
    print(text)
    break
    
    frequency = defaultdict(int)
    for token in text:
        frequency[token] += 1

    text = [token for token in text if frequency[token] > 1]
    print(text)
    
    # TODO feels like we need to integrate this with the thing above, since it won't line up otherwise
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    m = corpus2csc(corpus).transpose()
    m.get_shape()
    
    # todo rewrite this to make square with actual way we're doing this
    print(proba_output = model.predict_proba([d2v_model.infer_vector(row[0])]))